<font size = 6>$\Delta$CoVaR Data Preparation </font>
<font size = 4><div style="text-align: right"> Contributor: Haochen Jiang</div></font>
<font size = 4><div style="text-align: right"> May 6, 2022</div></font>

In [1]:
import math
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

from datetime import datetime

<font size = 4>Please make sure to input your Data path in your computer in the following format</font>

In [3]:
path = "D:\\Data\\"

<font size = 4>Please make sure to research time period in your computer in the following format:\
By the way, however, S&P 500 real estate sector index data before 2002/2/11 and after 2021/12/31 is not available!</font>

In [4]:
start = dt.datetime(year=2002, month=12, day=31)
end = dt.datetime(year=2021, month=12, day=31)

<font size = 4>Linux Debug Environment</font>

In [5]:
#! pip install nlopt
#! pip install yfinance
#! pip install pandas_datareader

In [6]:
#path = "/home/jovyan/demo/Data/"

# Load Data

<font size = 4>Data Sources:\
#1. _updated_A_snp_tickers_sectors.csv
It is found at https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Last change was on April 21, 2022.
\
#2. _updated_B_DTB3.csv (First state variable, Second state variable)
It is found at https://fred.stlouisfed.org/series/DTB3
Last change was on April 21, 2022.
\
#3. _updated_B_DLTBOARD.csv (Second state variable - first period)
It is found at https://fred.stlouisfed.org/series/DLTBOARD
Last change was on June 30, 2000.
\
#4. _updated_C_USTREASURY-LONGTERMRATES.csv (Second state variable - second period)
It is found at https://data.nasdaq.com/data/USTREASURY/LONGTERMRATES-treasury-long-term-rates
Last change was on January 31, 2022.
\
#5. _updated_B_TEDRATE.csv (Third state variable)
It is found at https://fred.stlouisfed.org/series/TEDRATE
Last change was on January 21, 2022.
\
#6. _updated_B_DBAA.csv (Fourth state variable)
It is found at https://fred.stlouisfed.org/series/DBAA
Last change was on April 21, 2022.
\
#7. _updated_B_DGS10.csv (Fourth state variable)
It is found at https://fred.stlouisfed.org/series/DGS10
Last change was on April 21, 2022.</font>

In [7]:
def interpolate(df):
    df.replace('.', np.nan, inplace = True)
    df = df.astype("float")

    missings = df[df.iloc[:, 0].isna()]
    front_temp = (missings.index + dt.timedelta(days=1)).isin(list(df.index))
    back_temp = (missings.index - dt.timedelta(days=1)).isin(list(df.index))
    both = front_temp & back_temp
    front = front_temp & ~back_temp
    back = ~front_temp & back_temp
    abnorm = ~front_temp & ~back_temp

    df.loc[missings[back].index] = df.loc[missings[back].index - dt.timedelta(days = 1)].values
    df.loc[missings[front].index] = df.loc[missings[front].index + dt.timedelta(days = 1)].values
    df.loc[missings[both].index] = (
        df.loc[missings[both].index + dt.timedelta(days=1)].values + df.loc[missings[both].index - dt.timedelta(days=1)].values) / 2

    if True not in abnorm:
        print("No abnormal value. Nothing abnormal has changed!")
    else:
        print("There are %d abnormal values!" % missings[abnorm].shape[0])
        for i in list(missings[abnorm].index):
            df["prev"] = df.iloc[:, 0].shift()
            df.loc[i, df.columns[0]] = df.loc[i, "prev"]
            while math.isnan(df.loc[i, df.columns[0]]) == True:
                df["prev"] = df["prev"].shift()
                df.loc[i, df.columns[0]] = df.loc[i, "prev"] 
            df.drop(columns = ["prev"], inplace = True)

    return df

## Load Stock Information Data 

In [8]:
# Load S&P 500 Sector Information
sector = pd.read_csv(path + "_updated_A_snp_tickers_sectors.csv", index_col = 0)
print("Number of stocks in S&P500: %d" % sector.shape[0])
# Make some necessary changes
sector.rename(index = {"BRK.B": "BRK-B", "BF.B": "BF-B"}, inplace=True)

Number of stocks in S&P500: 504


## Download Data (Required by first time)

In [9]:
# Create a New Directory in Windows Shell Command
# Please make sure to be consistent with your input path!
# !md "D:\\Data\\stocks"

In [10]:
# # Download S&P 500 Data
# system = yf.download('^GSPC', progress=False)
# realEstate = yf.download('^SP500-60', progress=False)
# financials = yf.download('^SP500-40', progress=False)
# system.to_csv(path + "stocks\\" + "_SNP500.csv")
# realEstate.to_csv(path + "stocks\\" + "_SNP500_realEstate.csv")
# financials.to_csv(path + "stocks\\" + "_SNP500_financials.csv")

In [11]:
%%time
# # Download Stocks Data
# bad_names = []
# for stock in list(sector.index):
#    try:
#        stock_df = web.DataReader(stock, 'yahoo', start = dt.date(1970,1,1))
#        stock_df['Name'] = stock
#        output_name = path + "stocks\\" + stock + ".csv"
#        stock_df.to_csv(output_name)
#    except:
#        bad_names.append(stock)
#        print('Data Not Found in Yahoo: %s.' % (stock))
#        #sector.drop(sector[sector.index == stock].index, inplace = True)
#        #print('Remove %s info from sector.' % (stock))
#
# print("Stocks Data Downloading Ends.")
# if len(bad_names) == 0:
#     print("All Stocks data are found.")
# else:
#     print("Number of stocks now in S&P500: %d" % sector.shape[0])

Wall time: 0 ns


## Load Market & Stocks prices Data 

In [12]:
# Load S&P 500 Data
system = pd.read_csv(path + "stocks\\" + "_SNP500.csv", index_col = 0, parse_dates = True)
# Load S&P 500 Real Estate Sector Data
realEstate = pd.read_csv(path + "stocks\\" + "_SNP500_realEstate.csv", index_col = 0, parse_dates = True)
# Load S&P 500 Financials Sector Data
financials = pd.read_csv(path + "stocks\\" + "_SNP500_financials.csv", index_col = 0, parse_dates = True)

# Load Stocks prices Data
stock_data = {}
for stock in list(sector.index):
    stock_df = pd.read_csv(path + "stocks\\" + stock + ".csv", index_col = 0, parse_dates = True)
    stock_data[stock] = stock_df

## Load State Variables Data 

In [13]:
# pd.set_option('display.max_rows', None)

### (i) Three-month yield Change
The author use the change in the three-month Treasury bill rate because the author finds that the change, not the level, is most significant in explaining the tails of financial sector market-valued asset returns

In [14]:
DTB3 = pd.read_csv(path + "_updated_B_DTB3.csv", index_col = 0, parse_dates = True)
DTB3 = interpolate(DTB3) / 100
DTB3_change = DTB3.loc[:,"DTB3"].diff().dropna().rename("3MO_Yield_Change")

No abnormal value. Nothing abnormal has changed!


### (ii) Term spread Change
Measured by the yield spread between the long term bond composite and the three-month bill rate

In [15]:
EARLY_LONGRATES = pd.read_csv(path + "_updated_B_DLTBOARD.csv", index_col = 0, parse_dates = True)
LATE_LONGRATES = pd.read_csv(path + "_updated_C_USTREASURY-LONGTERMRATES.csv", index_col = 0, parse_dates = True)

In [16]:
EARLY_LONGRATES = interpolate(EARLY_LONGRATES)
EARLY_LONGRATES.rename({"DLTBOARD": "LONGRATES"}, axis = 1, inplace = True)
LATE_LONGRATES.drop(columns = ["Treasury 20-Yr CMT", "Extrapolation Factor"], inplace = True)
LATE_LONGRATES.rename({"LT Composite > 10 Yrs": "LONGRATES"}, axis = 1, inplace = True)
LATE_LONGRATES = LATE_LONGRATES[LATE_LONGRATES.index > EARLY_LONGRATES.index[-1]]
LATE_LONGRATES.sort_index(ascending = True, inplace = True)
LONGRATES = pd.concat([EARLY_LONGRATES, LATE_LONGRATES], axis = 0, join="outer") / 100

No abnormal value. Nothing abnormal has changed!


In [17]:
temp1 = pd.concat([DTB3, LONGRATES], axis = 1, join="outer")
temp1 = temp1[(temp1.index >= LONGRATES.index[0]) & (temp1.index <= LONGRATES.index[-1])]
temp1.iloc[:,0] = interpolate(pd.DataFrame(temp1.iloc[:,0])).values
temp1.iloc[:,1] = interpolate(pd.DataFrame(temp1.iloc[:,1])).values

No abnormal value. Nothing abnormal has changed!
No abnormal value. Nothing abnormal has changed!


In [18]:
TERM_spread = temp1.loc[:,"DTB3"] - temp1.loc[:,"LONGRATES"]
TERM_spread_change = TERM_spread.diff().dropna().rename("Term_Spread_Change")

### (iii) TED spread
Defined as the difference between the three-month Libor rate and the three-month secondary market bill rate

In [19]:
TED_spread = pd.read_csv(path + "_updated_B_TEDRATE.csv", index_col = 0, parse_dates = True)
TED_spread = interpolate(TED_spread) / 100
TED_spread = TED_spread.loc[:,"TEDRATE"].rename("TED_Spread")

No abnormal value. Nothing abnormal has changed!


### (iv) Credit spread Change
Between Moody's Baa-rated bonds and the ten year Treasury rate

In [20]:
DBAA = pd.read_csv(path + "_updated_B_DBAA.csv", index_col = 0, parse_dates = True)
DBAA = interpolate(DBAA) / 100
DGS10 = pd.read_csv(path + "_updated_B_DGS10.csv", index_col = 0, parse_dates = True)
DGS10 = interpolate(DGS10) / 100

No abnormal value. Nothing abnormal has changed!
No abnormal value. Nothing abnormal has changed!


In [21]:
temp2 = pd.concat([DBAA, DGS10], axis = 1, join="outer")
temp2 = temp2[(temp2.index >= DBAA.index[0])]
CREDIT_spread = temp2.loc[:,"DBAA"] - temp2.loc[:,"DGS10"]
CREDIT_spread_change = CREDIT_spread.diff().dropna().rename("Credit_Spread_Change")

### (v) Market Return

In [22]:
market_close = system['Adj Close']
market_return = (market_close - market_close.shift(1)) / market_close.shift(1)
market_return = market_return.dropna().rename("Market_Return")

### (vi) Real Estate Excess Return
Measured by real estate sector return in excess of the market financial sector return

In [23]:
realEstate_close = realEstate['Adj Close']
realEstate_return = (realEstate_close - realEstate_close.shift(1)) / realEstate_close.shift(1)
realEstate_return = realEstate_return.dropna().rename("RealEstate_Return")
financials_close = financials['Adj Close']
financials_return = (financials_close - financials_close.shift(1)) / financials_close.shift(1)
financials_return = financials_return.dropna().rename("Financials_Return")

In [24]:
temp3 = pd.concat([realEstate_return, financials_return], axis = 1, join="outer")
temp3 = temp3[(temp3.index >= realEstate_return.index[0])]
excess_return = temp3.loc[:,"RealEstate_Return"] - temp3.loc[:,"Financials_Return"]
excess_return = excess_return.rename("Excess_Return")

### (vii) Equity Volatility
It can be computed as the 22 day rolling standard deviation of the daily CRSP equity market return

In [25]:
equity_volatility = market_return.rolling(22).std()
equity_volatility = equity_volatility.dropna().rename("Equity_Volatility")

### State Variables Summary
The spreads and spread changes are expressed in weekly basis points, and returns are in weekly percent

In [26]:
stateVariables_D = pd.concat(
    [
        DTB3_change,
        TERM_spread_change,
        TED_spread,
        CREDIT_spread_change,
        market_return,
        excess_return,
        equity_volatility,
    ],
    axis=1,
    join="outer",
)
stateVariables_D = stateVariables_D[(stateVariables_D.index >= start) & (stateVariables_D.index <= end)]
stateVariables_D = stateVariables_D.fillna(method = "backfill")

In [27]:
DTB3_change_W_bp = stateVariables_D.loc[:, "3MO_Yield_Change"].resample("W").sum() * 10000
TERM_spread_change_W_bp = stateVariables_D.loc[:, "Term_Spread_Change"].resample("W").mean() * 10000
TED_spread_W_bp = stateVariables_D.loc[:, "TED_Spread"].resample("W").mean() * 10000
CREDIT_spread_change_W_bp = stateVariables_D.loc[:, "Credit_Spread_Change"].resample("W").mean() * 10000
market_return_W_pct = ((stateVariables_D.loc[:, "Market_Return"] + 1).resample("W").prod() - 1) * 100
excess_return_W_pct = ((stateVariables_D.loc[:, "Excess_Return"] + 1).resample("W").prod() - 1) * 100
equity_volatility_W_pct = stateVariables_D.loc[:, "Equity_Volatility"].resample("W").max() * 100

stateVariables_W = pd.concat(
    [
        DTB3_change_W_bp,
        TERM_spread_change_W_bp,
        TED_spread_W_bp,
        CREDIT_spread_change_W_bp,
        market_return_W_pct,
        excess_return_W_pct,
        equity_volatility_W_pct,
    ],
    axis=1,
    join="outer",
)

In [28]:
stateVariables_W
stateVariables_W.describe().iloc[[1,2,3,4,6,7], :]

,3MO_Yield_Change,Term_Spread_Change,TED_Spread,Credit_Spread_Change,Market_Return,Excess_Return,Equity_Volatility
2003-01-05,0.0,-5.0,18.25,-2.25,6.750725,-3.224556,1.328661
2003-01-12,-2.0,-2.4,20.60,-1.60,2.088949,-4.375004,1.452672
2003-01-19,-2.0,2.2,20.00,-0.40,-2.780381,-2.196587,1.375343
2003-01-26,-2.0,1.6,20.00,0.80,-5.977730,4.543715,1.423311
2003-02-02,2.0,0.0,18.60,-3.40,-0.661715,2.086821,1.564426
...,...,...,...,...,...,...,...
2021-12-05,0.0,2.6,12.00,-0.20,-1.222951,2.054350,0.901189
2021-12-12,0.0,-3.4,13.60,0.40,3.824887,0.099011,1.058448
2021-12-19,-1.0,1.0,15.80,0.80,-1.939293,2.856688,1.154123
2021-12-26,3.0,-1.2,14.00,-0.20,3.691043,0.416471,1.296710


,3MO_Yield_Change,Term_Spread_Change,TED_Spread,Credit_Spread_Change,Market_Return,Excess_Return,Equity_Volatility
mean,-0.114919,0.033065,40.405595,-0.033317,0.218303,0.190356,1.031359
std,8.236839,2.602870,41.010937,1.709793,2.518130,3.662225,0.733573
min,-80.000000,-14.600000,6.600000,-7.400000,-18.195465,-18.888800,0.233922
25%,-1.000000,-1.400000,20.200000,-0.800000,-0.838042,-1.454589,0.623337
75%,2.000000,1.600000,42.600000,0.600000,1.470086,1.742171,1.189720
max,69.000000,11.600000,412.000000,18.600000,13.106071,19.366352,5.960092


## Measuring Losses

### Market Losses

In [29]:
market_losses = (market_close.shift(1) - market_close) / market_close.shift(1)
market_losses = market_losses[(market_losses.index >= start) & (market_losses.index <= end)].dropna()
market_losses_W_pct = (100 * (1 - (1 - market_losses).resample("W").prod())).rename("Market_Losses")
market_losses_W_pct = pd.DataFrame(market_losses_W_pct)

In [30]:
market_losses_W_pct.describe().iloc[[1,2,3,4,6,7], :]

,Market_Losses
mean,-0.199191
std,2.381427
min,-12.101749
25%,-1.419585
75%,0.815965
max,18.195465


### Stocks Losses

In [31]:
stocks_losses_W_pct = {}
for stock in stock_data.keys():
    stock_close = stock_data[stock]['Adj Close']
    stock_losses = (stock_close.shift(1) - stock_close) / stock_close.shift(1)
    stock_losses = stock_losses[(stock_losses.index >= start) & (stock_losses.index <= end)].dropna()
    stocks_losses_W_pct[stock] = (100 * (1 - (1 - stock_losses).resample("W").prod()))
stocks_losses_W_pct = pd.DataFrame(stocks_losses_W_pct)
stocks_losses_W_pct = stocks_losses_W_pct.loc[: ,(stocks_losses_W_pct.isna().sum(axis=0) < 750)]

In [32]:
stocks_losses_W_pct.describe()

,MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,...,WMB,WTW,WYNN,XEL,XYL,YUM,ZBRA,ZBH,ZION,ZTS
count,992.000000,992.000000,992.000000,470.000000,992.000000,992.000000,992.000000,992.000000,992.000000,992.000000,...,992.000000,992.000000,992.000000,992.000000,534.000000,992.000000,992.000000,992.000000,992.000000,465.000000
mean,-0.199544,-0.419284,-0.303941,-0.443523,-0.714064,-0.407798,-0.478494,-0.291316,-0.471637,-0.299679,...,-0.500489,-0.217582,-0.495617,-0.291759,-0.387933,-0.369603,-0.429062,-0.198189,-0.274264,-0.504537
std,2.912554,4.256573,2.914373,3.927986,7.074865,3.457911,4.517324,3.881790,4.126348,2.998530,...,5.697666,3.354330,7.124722,2.498282,3.500091,3.402942,4.686383,3.845529,6.524991,3.048484
min,-14.750158,-25.440581,-12.153672,-18.295588,-35.607943,-16.937248,-20.948062,-18.576083,-16.315626,-17.240762,...,-37.692362,-24.781986,-65.072373,-16.077419,-14.611565,-26.966832,-22.137207,-19.943480,-90.102764,-12.337504
25%,-1.887679,-2.691500,-1.864045,-2.463335,-3.962097,-2.211375,-3.024036,-2.330600,-2.758453,-1.882372,...,-3.054932,-1.897697,-3.620902,-1.681247,-2.289957,-2.274959,-2.920982,-2.192487,-2.634621,-2.176463
50%,-0.279004,-0.416808,-0.248270,-0.434629,-0.389338,-0.549010,-0.446563,-0.373434,-0.405585,-0.365615,...,-0.422870,-0.197354,-0.297814,-0.412523,-0.665449,-0.373939,-0.380421,-0.222093,-0.267681,-0.663276
75%,1.234264,1.651926,1.369727,1.585600,3.043722,1.431864,1.962418,1.729511,1.749028,1.239161,...,2.111001,1.391660,2.890544,1.064885,1.270873,1.437903,2.217298,1.666345,2.323770,1.282857
max,16.006208,19.995762,16.717678,19.386207,30.102969,16.567493,23.443226,19.055919,19.501330,22.449820,...,27.071470,20.723672,36.387347,24.251930,16.957845,25.975029,22.754254,24.252032,30.843141,19.556899
